In [130]:
import time
import  pickle
import re
import copy

import urllib.request as urllib2
from urllib import parse
from  bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [131]:
URL="https://www.zujuan.com/question?chid=6&xd=2&tree_type=knowledge"
#https://www.zujuan.com/question?chid=6&xd=2&tree_type=knowledge    #初中物理
#https://www.zujuan.com/question?chid=3&xd=2&tree_type=knowledge   #初中数学
#启用代理
proxy = urllib2.ProxyHandler({'http': '192.168.1.1'})
opener = urllib2.build_opener(proxy)
urllib2.install_opener(opener)
#伪装成浏览器
headers = {
    'User-Agent':'Mozilla/5.0 (Windows; U; Windows NT 6.1; en-US; rv:1.9.1.6) Gecko/20091201 Firefox/3.5.6'
}
request = urllib2.Request(
    url = URL,
    headers = headers
)
# url= "https://www.zujuan.com/question?chid=3&xd=2&tree_type=knowledge"
# url="https://www.baidu.com"
#获取到页面对象
respons = urllib2.urlopen(request)

In [132]:
def open_driver():
    URL="https://www.zujuan.com/question?chid=7&xd=2&tree_type=knowledge"
    try:
        driver.quit()
        print("正在打开浏览器……")
    except:
        print("正在打开浏览器……")
    driver = webdriver.Chrome()#打开chrome浏览器
    driver.maximize_window()
    driver.get(URL)#打开网址,方两个才可打开指定的，原因未知。。。
    driver.get(URL)#打开网址
    driver.implicitly_wait(0.5)#程序隐式等待1秒
    print("浏览器已打开！")
    return driver

In [133]:
def click_knowledge():
    time_start=time.time()
    first_list =driver.find_element_by_id('J_Tree').find_elements_by_tag_name("li")#获取到第一级列表
    knowledge_points = []#将要点击的最小知识点位置保存成list
    outline = {}#将大纲形成字典
    all_point_num = 0
    for first in first_list:
        click_first = first.find_element_by_tag_name('span')#第一级可点击元素
        click_first.click()#点击该元素，打开下一级
        element = WebDriverWait(driver, 6).until(#显式等待页面加载6秒
                EC.visibility_of(first.find_element_by_class_name('t2bd'))
        )
        second_list = first.find_element_by_class_name('t2bd').find_elements_by_tag_name("li")#获取到第二级列表
        first_text = first.find_element_by_class_name('t2hd').text#获取到第一级text
        outline[first_text]={}
        for second in second_list:
            click_second = second.find_element_by_tag_name('span')
            click_second.click()
            element = WebDriverWait(driver, 6).until(#显式等待页面加载6秒
                EC.visibility_of(second.find_element_by_tag_name("li"))
            )
            third_list = second.find_elements_by_tag_name("li")
            second_text = second.find_element_by_class_name('t2hd').text
            outline[first_text][second_text] ={}
            for third in third_list:
                third_text = third.find_element_by_tag_name('em').text
                third_click = third.find_element_by_tag_name('div').find_element_by_tag_name("em")
    #             third_click.click()
    #             time.sleep(3)
                knowledge_points.append(third_click)
                outline[first_text][second_text][third_text]={"click":third_click}
                all_point_num+=1
    #         break
    #     break
    # print(outline)
    time_end=time.time()
    print('点击耗时：',time_end-time_start)
    return outline,all_point_num

In [134]:
all_point_num

324

In [135]:
def save_file(outline):
    filename = "test.pkl"
    with open(filename,'wb+') as f:
        pickle.dump(outline, f, pickle.HIGHEST_PROTOCOL)

In [136]:
#将题目中图片位置换成 <!!mathml!!> 或者 <@@src@@>
def get_mathml_problem(problem,one_problem):
    one_problem_list = one_problem.split("<!!img!!>")#将本条数据字符串根据<!!img!!>分解为list
    problem_imgs = problem.find_all("img")#找到本条数据中全部的img
    for n in range(0, range(0,len(problem_imgs)).__len__())[::-1]:#逆序逐个将 <!!img!!> 替换为 <!!mathml语言!!> 
        rg = re.compile("<math.*?</math>",re.IGNORECASE|re.DOTALL)#寻找url地址解码后的mathml正则
        search_mathml = rg.search(parse.unquote(str(problem_imgs[n].attrs["src"])))#寻找url地址解码后的mathml
        if search_mathml :
            problem_imgs_mathml = search_mathml.group(0)#取出匹配的mathml
            one_problem_list.insert(n+1,"<!!"+str(problem_imgs_mathml)+"!!>")#将 <!!img!!> 替换为 <!!mathml语言!!> 
        else:
            one_problem_list.insert(n+1,"<@@"+str(problem_imgs[n].attrs["src"])+"@@>")#将 <!!img!!> 替换为 <!!mathml语言!!> 
    one_problem=''.join(one_problem_list)#将list转为str
#     print(one_problem)
    return one_problem

In [137]:
def type_content(every,every_topic):
    all_classify = every.select("> .exam-head > .exam-head-left")
    for classify in all_classify:#每个题目的 题型 题类 难易度
        classify = classify.stripped_strings
        for (index,val) in enumerate(classify):
            every_topic[classify_name[index]] = val.split("：")[-1]
    return every_topic

In [138]:
def topic_content(every,every_topic):
    #题目内容
    all_exercise_problems = every.select("> .exam-con")
    for exercise_problems in all_exercise_problems:#每个题目内容

        #提干处理
        list_exercise_problems=[]
        one_exercise_problem = ''#有的题目会被分成多行，用该变量重组
        for exercise_problem in exercise_problems.select(".exam-q"):#提干
            for problem in exercise_problem.children:
                problem = re.sub("(<img[^>]+>)","<!!img!!>", str(problem))#替换img标签为<!!img!!>（标记图片位置）
                if problem:
                    if problem!=" "  and problem!=None:
                        one_exercise_problem+=problem.strip()#将分散的题干整合到一起
            if exercise_problem.img != None:
                one_exercise_problem = get_mathml_problem(exercise_problem,one_exercise_problem)
        list_exercise_problems.append(one_exercise_problem)

        #选项处理
        all_option_list = []
        for all_options in exercise_problems.select(".exam-s .op-item"):#每个题的所有选项
            one_option = ''#有的选项会被分成多行，用该变量重组
            for options in all_options.children:#每个选项
                for option in options:
                    option = re.sub("(<img[^>]+>)","<!!img!!>", str(option))#替换img标签为<!!img!!>（标记图片位置）
                    if option:
                        if option!=" " and option!=None:
                            one_option+=str(option)
            if all_options.img != None:
                one_option = get_mathml_problem(all_options,one_option)#将题目中图片位置换成 <!!mathml!!> 或者 <@@src@@>
            all_option_list.append(one_option)
        list_exercise_problems.append(all_option_list)
        every_topic["topic"] = list_exercise_problems#将每个题目内容存入dict
    return every_topic

In [139]:

def run():
    time_start=time.time()
    first_num= 1
    second_num= 1
    third_num = 1
    this_point_num = 0
    classify_name=["type","class","diff"]
    for first_key,first_val in outline.items():
        for second_key,second_val in first_val.items():
            for third_key,third_val in second_val.items():
#                 #断点重连判断
#                 if continue_t[0]!="T":
#                     print(continue_t)
#                     print(first_key,second_key,third_key)
#                     print("@@@@@@@")
#                     time.sleep(1)
#                     if first_key == continue_t[1] and second_key == continue_t[2] and third_key == continue_t[3]:
#                         continue_t[0] = "T"
#                         print("！！！断点重连成功！！！")
#                     else:
#                         continue
#                 else:
#                     continue_t[-1] = 1
                if "click" in third_val:
                    try:
                        element = WebDriverWait(driver, 10).until(#显式等待将要点击的元素6秒
                                EC.visibility_of(third_val['click'])
                        )
                        third_val['click'].click()
                        time.sleep(2)
                    except:
    #                     driver.quit()
                        print(str(third_key)+" 点击出现问题")
                    this_point_num+=1#进度计数
                    third_val = []#将最后一级内容 从可点击对象变为list
                    #翻页
                    print(" 正在抓取 "+first_key+"->"+second_key+"->"+third_key+"**********总进度==>"+str((round(this_point_num/all_point_num,3)*100))+"%" )
                    for num in range(1,11):
#                         continue_t = ["T",first_key,second_key,third_key,num+1]
                        print("\r" + "* 第"+str(num)+"页 *", end='')
                        soup = BeautifulSoup(driver.page_source,"html.parser")
                        element = WebDriverWait(driver, 6).until(#显式等待页面加载6秒
                                EC.presence_of_element_located((By.ID,"J_QuestionList"))
                        )

                        #所有的题目
                        all_q = soup.select('#J_QuestionList li .search-exam')# > .exam-head > .exam-head-left
                        for every in all_q:#每个题目
                            every_topic = {}#存放每个题目
                            #题目类型处理
                            every_topic = type_content(every,every_topic)
                            #题目内容处理
                            every_topic = topic_content(every,every_topic)
                            third_val.append(every_topic)#将每个题目
                        outline[first_key][second_key][third_key] = third_val
                        try:
                            element = WebDriverWait(driver, 10).until(#显式等待页面加载10秒
                                EC.visibility_of(driver.find_element_by_css_selector('a[data-page="'+str((num+1))+'"]'))
                            )
                            time.sleep(0.6)
                            driver.find_element_by_css_selector('a[data-page="'+str((num+1))+'"]').click()#下一页
                            time.sleep(0.6)
                        except:
                            print("@未找到下一页,将抓取下个知识点@")
                            break
                    print("**ok*")
                else:
                    print("已经ok!")
#     #             break
#                 if third_num==2:
#                     third_num=0
#                     print(1+"1")
#                     break
#                 else:
#                     third_num+=1
#     #         break
#             if second_num==2:
#                 break
#             else:
#                 second_num+=1
#         if first_num==2:
#             break
#         else:
#             first_num+=1
#         except:
#             print("爬取错误！")
    time_end=time.time()
    print('总耗时：',time_end-time_start)
    

In [140]:
try:
    driver = open_driver()
    outline,all_point_num = click_knowledge()
#     continue_t = ["T",1]
    run()
except:
    print("！！！运行出现问题！！！")
#     driver.quit()
#     driver = open_driver()
#     outline_other,all_point_num = click_knowledge()
#     continue_t[0]="F"
#     run(continue_t)

正在打开浏览器……
浏览器已打开！
点击耗时： 69.86825394630432
 正在抓取 科学探究->对科学探究的理解->科学探究的意义**********总进度==>0.6%
* 第7页 *@未找到下一页,将抓取下个知识点@
**ok*
 正在抓取 科学探究->对科学探究的理解->猜想与事实验证**********总进度==>1.2%
* 第10页 ***ok*
 正在抓取 科学探究->对科学探究的理解->科学探究的基本方法**********总进度==>1.9%
* 第10页 ***ok*
 正在抓取 科学探究->对科学探究的理解->科学探究的基本环节**********总进度==>2.5%
* 第10页 ***ok*
 正在抓取 科学探究->科学探究能力->测定空气里氧气含量的探究**********总进度==>3.1%
* 第10页 ***ok*
 正在抓取 科学探究->科学探究能力->制取气体的反应原理的探究**********总进度==>3.6999999999999997%
* 第10页 ***ok*
 正在抓取 科学探究->科学探究能力->质量守恒定律的实验探究**********总进度==>4.3%
* 第10页 ***ok*
 正在抓取 科学探究->科学探究能力->金属活动性的探究**********总进度==>4.9%
* 第10页 ***ok*
 正在抓取 科学探究->科学探究能力->探究酸碱的主要性质**********总进度==>5.6000000000000005%
* 第10页 ***ok*
 正在抓取 科学探究->科学探究能力->药品是否变质的探究**********总进度==>6.2%
* 第10页 ***ok*
 正在抓取 科学探究->科学探究能力->缺失标签的药品成分的探究**********总进度==>6.800000000000001%
* 第10页 ***ok*
 正在抓取 科学探究->科学探究能力->实验探究物质的性质或变化规律**********总进度==>7.3999999999999995%
* 第10页 ***ok*
 正在抓取 科学探究->科学探究能力->探究金属锈蚀的条件**********总进度==>8.0%
* 第10页 ***ok*
 正在抓取 科学探究->科学探究能力->燃烧的条件与灭火原理探

In [142]:
#{first:{second:{third:{
#     "topic":[[题目,选项],[题目,选项]],"diff":"难度","type":"题型","class":"题类"}
# }}}
# {"一级知识点":{"二级知识点":{"三级知识点":[
#     {'type': '填空题', 'class': '常考题', 'diff': '普通', 'topic': ['1. 如图所示的长方体，用符号表示下列棱的位置关系：A<sub>1</sub>B<sub>1</sub><span class="q-blank">\xa0</span>AB，AA<sub>1</sub><span class="q-blank">\xa0</span>BB<sub>1</sub>， A<sub>1</sub>D<sub>1</sub><span class="q-blank">\xa0</span>C<sub>1</sub>D<sub>1</sub>， AD<span class="q-blank">\xa0</span>BC．<br/><@@http://tikupic.21cnjy.com/c1/4e/c14e826d91fe0130c6b9bc1ff0af3eed.png@@>', []]}, 
#     {'type': '填空题', 'class': '常考题', 'diff': '普通', 'topic': ['2. 如图，在正方体ABCD﹣A′B′C′D′中，与棱AD平行的棱有<span class="q-blank">\xa0</span>条．<br/><@@http://tikupic.21cnjy.com/af/ea/afea87169f279692bcb04a882f161078.png@@>', []]}, {'type': '填空题', 'class': '常考题', 'diff': '普通', 'topic': ['3. 如图，在正方体中，与线段AB平行的线段有<span class="q-blank">\xa0</span>．<br/><@@http://tikupic.21cnjy.com/c2/2e/c22e0b5540bb7f1a238ec267dfafe18e.png@@>', []]},
#     {'type': '解答题', 'class': '常考题', 'diff': '普通', 'topic': ['4. 学校为实验教室配备了一只无盖的圆柱形铁皮消防桶．做这只消防桶至少需要铁皮多少平方分米？<@@http://tikupic.21cnjy.com/48/97/489702b59b482c8702ee55bd92a51a2b.png@@>', []]}, {'type': '单选题', 'class': '常考题', 'diff': '普通', 'topic': ['5. 若一个棱柱有10个顶点，则下列说法正确的是（\xa0\xa0 ）', ['A、这个棱柱有4个侧面', 'B、这个棱柱有5条侧棱', 'C、这个棱柱的底面是十边形', 'D、这个棱柱是一个十棱柱']]}, 
#     {'type': '解答题', 'class': '常考题', 'diff': '普通', 'topic': ['6. 如图所示的立方体的六个面分别标着连续的整数，求这六个整数的和．<@@http://tikupic.21cnjy.com/c3/00/c300666beff28f825675225416fc3799.png@@>', []]}, {'type': '综合题', 'class': '常考题', 'diff': '普通', 'topic': ['7. 一个圆柱体形的蓄水池，从里面量底面周长31.4米，深2.4米，在它的内壁与底面抹上水泥．（1）、抹水泥部分的面积是多少平方米？（2）、蓄水池能蓄水多少吨？（每立方米水重1千克）', []]}, 
#     {'type': '单选题', 'class': '常考题', 'diff': '普通', 'topic': ['8. 底面积相等的圆柱和圆锥，它们的体积比是1：2，圆锥的高是9厘米，圆柱的高是（\xa0\xa0 ）厘米．', ['A、3', 'B、1.5', 'C、18', 'D、24']]}, 
#     {'type': '判断题', 'class': '常考题', 'diff': '普通', 'topic': ['9. 棱柱侧面的形状可能是一个三角形.（判断对错）', []]}, 
#     {'type': '单选题', 'class': '常考题', 'diff': '普通', 'topic': ['10. 下列说法：①柱体的两个底面一样大；②圆柱、圆锥的底面都是圆；③棱柱的底面是四边形；④长方体一定是柱体；⑤直棱柱的侧面一定是长方形．其中正确的个数是（\xa0\xa0 ）', ['A、2个', 'B、3个', 'C、4个', 'D、5个']]}
#     {'type': '综合题', 'class': '常考题', 'diff': '普通', 'topic': ['2. 阅读理解：已知点P（x<sub>0</sub>， y<sub>0</sub>）和直线y=kx+b，则点P到直线y=kx+b的距离，可用公式d=<!!<math+xmlns="http://www.w3.org/1998/Math/MathML"><mrow><mfrac><mrow><mrow><mo>|</mo><mrow><mi>k</mi><msub><mi>x</mi><mn>0</mn></msub><mo>−</mo><msub><mi>y</mi><mn>0</mn></msub><mo>+</mo><mi>b</mi></mrow><mo>|</mo></mrow></mrow><mrow><msqrt><mrow><mn>1</mn><mo>+</mo><msup><mi>k</mi><mn>2</mn></msup></mrow></msqrt></mrow></mfrac></mrow></math>!!>计算．<br/>例如：求点P（﹣1，2）到直线y=3x+7的距离．<br/>解：因为直线y=3x+7，其中k=3，b=7．<br/>所以点P（﹣1，2）到直线y=3x+7的距离为：d=<!!<math+xmlns="http://www.w3.org/1998/Math/MathML"><mrow><mfrac><mrow><mrow><mo>|</mo><mrow><mi>k</mi><msub><mi>x</mi><mn>0</mn></msub><mo>−</mo><msub><mi>y</mi><mn>0</mn></msub><mo>+</mo><mi>b</mi></mrow><mo>|</mo></mrow></mrow><mrow><msqrt><mrow><mn>1</mn><mo>+</mo><msup><mi>k</mi><mn>2</mn></msup></mrow></msqrt></mrow></mfrac></mrow></math>!!>=<!!<math+xmlns="http://www.w3.org/1998/Math/MathML"><mrow><mfrac><mrow><mrow><mo>|</mo><mrow><mn>3</mn><mo>×</mo><mrow><mo>(</mo><mrow><mo>−</mo><mn>1</mn></mrow><mo>)</mo></mrow><mo>−</mo><mn>2</mn><mo>+</mo><mn>7</mn></mrow><mo>|</mo></mrow></mrow><mrow><msqrt><mrow><mn>1</mn><mo>+</mo><msup><mn>3</mn><mn>2</mn></msup></mrow></msqrt></mrow></mfrac></mrow></math>!!>=<!!<math+xmlns="http://www.w3.org/1998/Math/MathML"><mrow><mfrac><mn>2</mn><mrow><msqrt><mrow><mn>10</mn></mrow></msqrt></mrow></mfrac></mrow></math>!!>=<!!<math+xmlns="http://www.w3.org/1998/Math/MathML"><mrow><mfrac><mrow><msqrt><mrow><mn>10</mn></mrow></msqrt></mrow><mn>5</mn></mfrac></mrow></math>!!>．<br/>根据以上材料，解答下列问题：（1）、求点P（1，﹣1）到直线y=x﹣1的距离；（2）、已知⊙Q的圆心Q坐标为（0，5），半径r为2，判断⊙Q与直线y=<!!<math+xmlns="http://www.w3.org/1998/Math/MathML"><mrow><msqrt><mn>3</mn></msqrt></mrow></math>!!>x+9的位置关系并说明理由；（3）、已知直线y=﹣2x+4与y=﹣2x﹣6平行，求这两条直线之间的距离．', []]}, 
#     ………………
# ]}}}

# 图片格式：<@@图片地址@@>，例如<@@http://tikupic.21cnjy.com/af/ea/afea87169f279692bcb04a882f161078.png@@>
# 公式格式：<!!公式!!>，例如 <!!<math+xmlns="http://www.w3.org/1998/Math/MathML"><mrow><mfrac><mrow><mrow><mo>|</mo><mrow><mi>k</mi><msub><mi>x</mi><mn>0</mn></msub><mo>−</mo><msub><mi>y</mi><mn>0</mn></msub><mo>+</mo><mi>b</mi></mrow><mo>|</mo></mrow></mrow><mrow><msqrt><mrow><mn>1</mn><mo>+</mo><msup><mi>k</mi><mn>2</mn></msup></mrow></msqrt></mrow></mfrac></mrow></math>!!>

# for n in range(1, range(0,2+1).__len__())[::-1]:
#     print (n)

outline["身边的化学物质"]["水与常见的溶液"]["有关溶质质量分数的简单计算"]
# outline["数与式"]["无理数与实数"]["有理数的乘法运算律"]

[{'class': '真题',
  'diff': '普通',
  'topic': ['1. KNO<sub>3</sub>与KCl的溶解度曲线如右图所示。下列说法正确的是（\xa0\xa0 ）<br/><@@http://tikupic.21cnjy.com/e6/ba/e6ba55081171afd19b2bb6238596aca7.jpg@@>',
   ['A、KNO<sub>3</sub>的溶解度比KCl的溶解度大',
    'B、t<sub>1</sub>温度下的KCl的饱和溶液升温至t<sub>2</sub>  ， 有晶体析出',
    'C、A点所表示的KCl饱和溶液溶质的质量分数为29%',
    'D、冷却热的KNO<sub>3</sub>饱和溶液可获得KNO<sub>3</sub>晶体']],
  'type': '单选题'},
 {'class': '真题',
  'diff': '普通',
  'topic': ['2. 实验室用68g过氧化氢溶液和2g二氧化锰制取氧气,实验的相关数据如图。<br/><@@http://tikupic.21cnjy.com/29/f7/29f718821924f37b22693dfbc26f7425.png@@><br/>请回答:（1）、二氧化锰作为催化剂在化学反应前后本身的<span class="q-blank">\xa0</span>都没有发生变化；（2）、反应生成氧气的质量为<span class="q-blank">\xa0</span>g。（3）、参加反应过氧化氢溶液的质量分数是<span class="q-blank">\xa0</span>?',
   []],
  'type': '计算题'},
 {'class': '真题',
  'diff': '普通',
  'topic': ['3. 某工厂利用废硫酸与废铁屑反应制取硫酸亚铁。现取废硫酸4.9 t 与足量的废铁屑反应，得到FeSO<sub>4</sub>1.52 t。计算废硫酸中H<sub>2</sub>SO<sub>4</sub>的质量分数为多少。',
   []],
  'type': '计算题'},
 {'class': '真题',
  'diff': '普通',
  'topic': ['4. 下图是甲、乙两种固体

In [144]:
#清空字典内的对象，以便保存进文件。
for first_key,first_val in outline.items():
    for second_key,second_val in first_val.items():
        for third_key,third_val in second_val.items():
            if "click" in third_val:
                outline[first_key][second_key][third_key]=[]
    print("END BIG DATA")

filename = "huaxue_data_56.pkl"
with open(filename,'wb+') as f:
    pickle.dump(outline, f, pickle.HIGHEST_PROTOCOL)
#     f.write(buf)
#     f.flush()
#     os.fsync(f.fileno)

In [22]:
# with open(filename, 'rb') as f:
#     x = pickle.load(f)
#     print(x["图形的性质"]["三角形"])

In [ ]:
next_url = 'https://www.zujuan.com'+soup.find(attrs={"data-page": "2"}).attrs["href"]
next_url

END BIG DATA
END BIG DATA
END BIG DATA
END BIG DATA


9
10
XX
